OpenAI provides many awesome APIs that help developers around the world to build their AI apps. One of the struggles many developers face is inconsistent output, which is not suitable for programmatic use.

* https://medium.com/@ismalinggazein/openai-function-calling-integrating-external-weather-api-6935e5d701d3
* https://medium.com/@igorcosta/beyond-basics-transforming-your-approach-with-advanced-openai-function-calls-and-tools-7e569f2427f0

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()


import requests
import json



In [2]:

from typing import Optional, Dict, Any

TMDB_API_KEY = os.getenv("TMDB_API_KEY")

if not TMDB_API_KEY:
    raise ValueError("TMDB_API_KEY environment variable is not set")

BASE_URL = "https://api.themoviedb.org/3"

def make_request(endpoint: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    url = f"{BASE_URL}{endpoint}"
    params = params or {}
    params['api_key'] = TMDB_API_KEY
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def get_movie_id(movie_name: str) -> Optional[int]:
    data = make_request("/search/movie", {"query": movie_name})
    
    if data['results']:
        return data['results'][0]['id']
    return None

def get_movie_details(movie_id: int) -> Optional[str]:
    data = make_request(f"/movie/{movie_id}")
    overview = data.get('overview')
    tagline = data.get('tagline')
    return {'overview': overview, 'tagline': tagline}

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_movie_detail",
            "description": "Get information about a movie",
            "parameters": {
                "type": "object",
                "properties": {
                    "overview": {
                        "type": "string",
                        "description": "Details about a movie e.g., Garfield"
                    },
                },
                "required": ["movie_name"],
            }
        }
    }
]

messages = [
    {"role": "user", "content": "Tell me something about ps i love you?"}]

In [4]:

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key= OPENAI_API_KEY)


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)

In [5]:
print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_DY8og1I9c997XzSfBQrr2Gaq', function=Function(arguments='{"overview":"PS I Love You is a romantic drama film based on the novel of the same name by Cecelia Ahern. The story follows Holly Kennedy, a young widow whose husband, Gerry, has left her a series of letters to help her cope with his death and navigate her new life without him."}', name='get_movie_detail'), type='function')])
